In [1]:
import os
# NOTE: material: https://drive.google.com/drive/u/0/folders/14J6oWZzULqtXfIv99064Cs0iHVdeat22

In [5]:
# NOTE: access to directory
dir_project = os.path.abspath(".")
print(f"{dir_project=}")
os.chdir(dir_project)
print(f"{os.getcwd()}")

dir_project='/root/Documents/Projects/augmented_safeguard/NeRF'
/root/Documents/Projects/augmented_safeguard/NeRF


In [ ]:
def config_parser():
    import argparse
    parser = argparse.ArgumentParser()

    parser.add_argument("--expname", type=str, help="experiment name")
    parser.add_argument("--basedir", type=str, default="./logs/", help="where to store checkpoints and logs")
    parser.add_argument("--datadir", type=str, default="./data/llff/fern", help="input data directory")

    # NOTE: tranining options
    # ------------------------------------------
    parser.add_argument("--netdepth", type=int, default=8, help="layers in network")
    parser.add_argument("--netwidth", type=int, default=256, help="channels per layer")
    parser.add_argument("--netdepth_fine", type=int, default=8, help="layers in fine network")
    parser.add_argument("--netwidth_fine", type=int, default=256, help="channels per layer in fine network")
    parser.add_argument("--N_rand", type=int, default=32*32*4, help="batch size (number of random rays per gradient step)")
    parser.add_argument("--lrate", type=float, default=5e-4, help="learning rate")
    parser.add_argument("--lrate_decay", type=int, default=250, help="exponential learning rate decay (in 1000 steps)")
    
    ## NOTE: training options - batch size
    parser.add_argument("--chunk", type=int, default=1024*32, help="number of rays processed in parallel, decrease it if running out of memory")
    parser.add_argument("--netchunk", type=int, default=1024*64, help="number of points sent through network in parallel, decrease it if running out of memory")
    parser.add_argument("--no_batching", action="store_true", help="only take random rays from 1 image at a time")
    parser.add_argument("--no_reload", action="store_true", help="do not reload weights from saved checkpoint")
    parser.add_argument("--ft_path", type=str, default=None, help="specific weights npy file to reload for coarse network")
    # ------------------------------------------


    # NOTE: rendering options
    
